In [108]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [109]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [110]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [111]:
metrics = loadMetricData('sarahYee', "metrics")
metrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-04 23:55:00,71,0,0.9,0.000067,93.2,93.2
1436,2016-05-04 23:56:00,69,0,1.0,0.000067,94.1,94.1
1437,2016-05-04 23:57:00,68,0,0.9,0.000069,94.1,94.1
1438,2016-05-04 23:58:00,66,0,1.0,0.000071,94.1,94.1
1439,2016-05-04 23:59:00,65,0,0.9,0.000067,94.1,94.1


In [112]:
metrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [113]:
# clean metric data

metrics.timestamp = pd.to_datetime(metrics.timestamp)

metrics.heartrate = metrics.heartrate.fillna(0)
metrics.steps = metrics.steps.fillna(0)
metrics.calories = metrics.calories.fillna(0)
metrics.gsr = metrics.gsr.fillna(0)
metrics.skintemp = metrics.skintemp.fillna(0)
metrics.airtemp = metrics.airtemp.fillna(0)

# metrics.head(5)

In [183]:
metrics = metrics[metrics.heartrate != 0]
metrics = metrics[metrics.skintemp != 0]
metrics = metrics[metrics.airtemp != 0]

metrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,xlt,xgt,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
706,2016-04-22 11:46:00,80,0,1.6,0.000054,89.6,88.7,False,False,True,...,False,False,False,False,False,False,False,False,False,False
710,2016-04-22 11:50:00,81,0,1.3,0.000064,88.7,87.8,False,False,True,...,False,False,False,False,False,False,False,False,False,False
711,2016-04-22 11:51:00,85,0,1.4,0.000067,90.5,87.8,False,False,True,...,False,False,False,False,False,False,False,False,False,False
712,2016-04-22 11:52:00,82,0,1.5,0.000066,90.5,87.8,False,False,True,...,False,False,False,False,False,False,False,False,False,False
713,2016-04-22 11:53:00,90,15,2.1,0.000060,90.5,88.7,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [184]:
# activities = loadMetricData('sarahYee', "activities")
# activities.tail(5)

In [185]:
# sleep = loadMetricData('sarahYee', "sleep")
# sleep.tail(5)

In [186]:
# sleep.dtypes

In [187]:
# journal = loadMetricData('sarahYee', "journal")
# journal.tail(5)

In [188]:
# journal.dtypes

In [189]:
# clean metric data
journal.starttime= pd.to_datetime(journal.starttime)
journal.endtime= pd.to_datetime(journal.endtime)

journal.qualitative = journal.qualitative.fillna(0)
journal.excited = journal.excited.fillna(0)
journal.happy = journal.happy.fillna(0)
journal.calm = journal.calm.fillna(0)
journal.anxious = journal.anxious.fillna(0)
journal.sad = journal.sad.fillna(0)
journal.angry = journal.angry.fillna(0)
journal.hungry = journal.hungry.fillna(0)
journal.tired = journal.tired.fillna(0)
journal.bored = journal.bored.fillna(0)
journal.ansaan = journal.ansaan.fillna(0)
journal.exhaca = journal.exhaca.fillna(0)

print(journal.shape)
journal.head(5)
# journal.tail(5)

(127, 33)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,False,False,True,False,False,False,False,False,False,False
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,5/3,5,...,True,True,False,False,True,False,False,False,False,False
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,False,False,False,False,False,False,False,False,False,False


In [190]:

journal['boolexcited'] =  journal['excited'] > 0
journal['boolhappy'] =  journal['happy'] > 0
journal['boolcalm'] =  journal['calm'] > 0
journal['boolanxious'] =  journal['anxious'] > 0
journal['boolsad'] =  journal['sad'] > 0
journal['boolangry'] =  journal['angry'] > 0
journal['booltired'] =  journal['tired'] > 0
journal['boolhungry'] =  journal['hungry'] > 0
journal['boolbored'] =  journal['bored'] > 0
journal['exhaca'] =  journal['exhaca'] > 0
journal['ansaan'] =  journal['ansaan'] > 0

journal['exha'] =  journal['exha'] > 0
journal['haca'] =  journal['haca'] > 0
journal['anan'] =  journal['anan'] > 0
journal['ansa'] =  journal['anan'] > 0

    
boolexcited = journal.iloc[:, 18].values.reshape(journal.shape[0], 1)
boolhappy = journal.iloc[:, 19].values.reshape(journal.shape[0], 1)
boolcalm = journal.iloc[:, 20].values.reshape(journal.shape[0], 1)
boolanxious = journal.iloc[:, 21].values.reshape(journal.shape[0], 1)
boolsad = journal.iloc[:, 22].values.reshape(journal.shape[0], 1)
boolangry = journal.iloc[:, 23].values.reshape(journal.shape[0], 1)
booltired = journal.iloc[:, 24].values.reshape(journal.shape[0], 1)
boolhungry = journal.iloc[:, 25].values.reshape(journal.shape[0], 1)
boolbored = journal.iloc[:, 26].values.reshape(journal.shape[0], 1)
exhaca = journal.iloc[:, 27].values.reshape(journal.shape[0], 1)
ansaan = journal.iloc[:, 28].values.reshape(journal.shape[0], 1)
exha = journal.iloc[:, 29].values.reshape(journal.shape[0], 1)
haca = journal.iloc[:, 30].values.reshape(journal.shape[0], 1)
anan = journal.iloc[:, 31].values.reshape(journal.shape[0], 1)
ansa = journal.iloc[:, 32].values.reshape(journal.shape[0], 1)

journal.head(5)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-04-22 13:48:00,2016-04-22 16:12:00,4/22/16,1:48:00 PM,4:12:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
1,2016-04-22 16:12:00,2016-04-22 17:15:00,4/22/16,4:12:00 PM,5:15:00 PM,writing thesis,barb's office,anxious,6,0,...,True,False,False,False,True,False,False,False,False,False
2,2016-04-22 17:15:00,2016-04-22 19:05:00,4/22/16,5:15:00 PM,7:05:00 PM,eating russian dumplings with annie then watch...,5th floor lounge,happy,10,0,...,False,False,True,False,False,False,False,False,False,False
3,2016-04-22 19:05:00,2016-04-22 19:15:00,4/22/16,7:05:00 PM,7:15:00 PM,photofeast,3rd floor,excited/anxious,5/3,5,...,True,True,False,False,True,False,False,False,False,False
4,2016-04-22 19:15:00,2016-04-22 19:25:00,4/22/16,7:15:00 PM,7:25:00 PM,naked picture of charles in photofeast,3rd floor,what can only be described as traumatic shock,10,0,...,False,False,False,False,False,False,False,False,False,False


In [191]:
# for i in range(0, sarahJournal.shape[0]):
#     sarahJournal['ex'] = (0< sarahJournal.iloc[i, 18].astype(int))
#     sarahJournal['ha'] = sarahJournal.iloc[i, 19]== True
#     sarahJournal['ca'] = sarahJournal.iloc[i, 20]== True
#     sarahJournal['ys'] = sarahJournal['ex'].astype(int) + sarahJournal['ha'].astype(int) + sarahJournal['ca'].astype(int)

# print(sarahJournal['ex'])
    

In [192]:
# def makeEmoIndicators(newvar, journalData, journal_emo1_loc, journal_emo2_loc, journal_emo3_loc):
  
#     journalData[newvar] = False
    
#     for i in range(0, journalData.shape[0]):
#         journalData['emo1'] = (journalData.iloc[i, journal_emo1_loc]) == True)
#         journalData['emo2'] = (journalData.iloc[i, journal_emo2_loc] == True)
#         journalData['emo3'] = (journalData.iloc[i, journal_emo3_loc] == True)
#         journalData['ysum'] = journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int)
# #         print(journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int))
#         journalData.loc[journalData['ysum']!=0, newvar] = True
        
#     print(journalData['emo1'].values)
#     journalData = journalData.drop(['emo1', 'emo2', 'emo3', 'ysum'], axis=1)
#     return journalData

In [193]:
# sarahJournal.dtypes

In [194]:
# testData= makeEmoIndicators('exhaca', sarahJournal, 18, 19, 20)
# # print(testData.describe())

In [195]:
# sarahJournal.boolexhapcal.convert_objects(convert_numeric=True)

In [196]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [197]:
# indicator for walking
totalData = makeIndicators('excitement', metrics, journal, 0, 1, 18)
totalData = makeIndicators('happy', metrics, journal, 0, 1, 19)
totalData = makeIndicators('calm', metrics, journal, 0, 1, 20)
totalData = makeIndicators('anxious', metrics, journal, 0, 1, 21)
totalData = makeIndicators('sad', metrics, journal, 0, 1, 22)
totalData = makeIndicators('angry', metrics, journal, 0, 1, 23)
totalData = makeIndicators('hungry', metrics, journal, 0, 1, 24)
totalData = makeIndicators('tired', metrics, journal, 0, 1, 25)
totalData = makeIndicators('bored', metrics, journal, 0, 1, 26)

totalData = makeIndicators('exhaca', metrics, journal, 0, 1, 27)
totalData = makeIndicators('ansaan', metrics, journal, 0, 1, 28)

totalData = makeIndicators('exha', metrics, journal, 0, 1, 29)
totalData = makeIndicators('haca', metrics, journal, 0, 1, 30)
totalData = makeIndicators('anan', metrics, journal, 0, 1, 31)
totalData = makeIndicators('ansa', metrics, journal, 0, 1, 32)




In [198]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  17223.000000  17223.000000  17223.000000  17223.000000  17223.000000   
mean      87.653951      8.782152      1.757017      0.045695     89.051054   
std       18.064607     25.965310      1.435048      0.418340      3.948830   
min       57.000000      0.000000      0.000000      0.000000     68.900000   
25%       71.000000      0.000000      1.000000      0.000054     86.900000   
50%       87.000000      0.000000      1.200000      0.000062     89.600000   
75%       99.000000      0.000000      1.800000      0.000081     91.400000   
max      168.000000    695.000000     11.200000     10.500000     97.700000   

            airtemp excitement     happy       calm   anxious    ...      \
count  17223.000000      17223     17223      17223     17223    ...       
mean      85.575579   0.184463  0.245544  0.0479591  0.178134    ...       
std        5.077088   0.387872  0.430422   0.213686  0.38263

In [199]:
print(totalData.head())

              timestamp  heartrate  steps  calories       gsr  skintemp  \
706 2016-04-22 11:46:00         80      0       1.6  0.000054      89.6   
710 2016-04-22 11:50:00         81      0       1.3  0.000064      88.7   
711 2016-04-22 11:51:00         85      0       1.4  0.000067      90.5   
712 2016-04-22 11:52:00         82      0       1.5  0.000066      90.5   
713 2016-04-22 11:53:00         90     15       2.1  0.000060      90.5   

     airtemp excitement  happy   calm  ...    angry hungry  tired  bored  \
706     88.7      False  False  False  ...    False  False  False  False   
710     87.8      False  False  False  ...    False  False  False  False   
711     87.8      False  False  False  ...    False  False  False  False   
712     87.8      False  False  False  ...    False  False  False  False   
713     88.7      False  False  False  ...    False  False  False  False   

    exhaca ansaan   exha   haca   anan   ansa  
706  False  False  False  False  False  Fals

In [200]:
totalData.to_csv('sarahTotal.csv')

In [201]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
706,2016-04-22 11:46:00,80,0,1.6,0.000054,89.6,88.7,False,False,False,...,False,False,False,False,False,False,False,False,False,False
710,2016-04-22 11:50:00,81,0,1.3,0.000064,88.7,87.8,False,False,False,...,False,False,False,False,False,False,False,False,False,False
711,2016-04-22 11:51:00,85,0,1.4,0.000067,90.5,87.8,False,False,False,...,False,False,False,False,False,False,False,False,False,False
712,2016-04-22 11:52:00,82,0,1.5,0.000066,90.5,87.8,False,False,False,...,False,False,False,False,False,False,False,False,False,False
713,2016-04-22 11:53:00,90,15,2.1,0.000060,90.5,88.7,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [202]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute

# print(totalData.shape)
# print(totalData.head(5))
# print(totalData['timestamp'].dt.minute)

In [203]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)

# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData['weekday'].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData['hour'].values.reshape(totalData.shape[0], 1)
curmin = totalData['min'].values.reshape(totalData.shape[0], 1)

print(hour)

[[11]
 [11]
 [11]
 ..., 
 [23]
 [23]
 [23]]


In [204]:
totalData.to_csv('newsarahTotal.csv')

In [205]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp,
        weekday
#         hour
#         curmin
    ), axis=1)

X.shape

(17223, 7)

# Positive ( Excited, Happy, Calm)

In [206]:
# define y


y = totalData.exhaca.values

# y = totalData.excitement.values
y.shape

(17223,)

In [207]:
# create training and test sets
from sklearn.cross_validation import train_test_split
#
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [208]:
# # feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [209]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

## Positive ( Excited, Happy, Calm) Model Results

In [210]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_train)
y_pred_test = clf.predict(X_test)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.363138686131 0.359783239791
 Accuracy Rate 0.636861313869 0.640216760209
True Positives  0.72860277981 0.731932773109
True Negatives             -- 0.612773447322


(0.3597832397909812,
 0.64021676020901874,
 0.73193277310924365,
 0.61277344732210204)

In [211]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50000, alpha=0.0001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_train)
y_pred_test = clf.predict(X_test)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.264764432648 0.270369653571
      Accuracy Rate 0.735235567352 0.729630346429
     True Positives 0.389173372348 0.385714285714
     True Negatives             -- 0.832537088257


(0.27036965357073739,
 0.72963034642926261,
 0.38571428571428573,
 0.83253708825748052)

In [181]:
print(clf.coef_)
print(clf.intercept_)
print(X_train_std[0:5])
print(X_train[0:5])
# heartrate, steps, calories, gsr, skintemp, airtemp, weekday, hour, curmin


[[ 0.85493352 -0.14848222 -3.17003065 -0.06323168  0.26426268 -0.58902858
   0.15321813]]
[-1.64868109]
[[  3.52759580e+00   0.00000000e+00   1.02846809e-01   1.47285951e-04
    1.46583562e+01   1.32235531e+01   2.83422731e+00]
 [  4.07878264e+00   0.00000000e+00   9.34970994e-02   1.54699673e-04
    1.47999068e+01   1.33524935e+01   2.83422731e+00]
 [  5.84258054e+00   4.60966496e-01   1.30895939e-01   1.57170914e-04
    1.42337042e+01   1.25788512e+01   4.72371218e-01]
 [  4.51973211e+00   0.00000000e+00   1.02846809e-01   1.24056288e-04
    1.21104445e+01   1.05158050e+01   9.44742436e-01]
 [  4.68508817e+00   0.00000000e+00   1.49595359e-01   1.41602097e-04
    1.33844003e+01   1.16762684e+01   9.44742436e-01]]
[[  6.40000000e+01   0.00000000e+00   1.10000000e+00   5.96000000e-05
    9.32000000e+01   9.23000000e+01   6.00000000e+00]
 [  7.40000000e+01   0.00000000e+00   1.00000000e+00   6.26000000e-05
    9.41000000e+01   9.32000000e+01   6.00000000e+00]
 [  1.06000000e+02   1.2000

In [172]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.239851178173  0.254822530864
 Accuracy Rate 0.760148821827  0.745177469136
True Positives 0.035237388724 0.0274636510501
True Negatives             --  0.970349721237


(0.25482253086419754,
 0.74517746913580241,
 0.027463651050080775,
 0.97034972123669538)

In [44]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.224803637867    0.24112654321
 Accuracy Rate   0.775196362133    0.75887345679
True Positives 0.00259643916914 0.00242326332795
True Negatives               --    0.99619868221


(0.24112654320987653,
 0.75887345679012341,
 0.0024232633279483036,
 0.99619868220983276)

# Negative ( Anxious, Sad, Angry)

In [45]:
# define y
y = totalData.ansaan.values
y.shape

(17279,)

In [46]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [47]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Negative ( Anxious, Sad, Angry) Model Results

In [48]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0884663083919 0.0887345679012
 Accuracy Rate  0.911533691608  0.911265432099
True Positives  0.292775665399  0.269058295964
True Negatives              --  0.940133037694


(0.088734567901234573,
 0.91126543209876543,
 0.26905829596412556,
 0.94013303769401335)

In [49]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0434890450599 0.0430169753086
      Accuracy Rate   0.95651095494  0.956983024691
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.043016975308641972, 0.95698302469135799, 0.0, 1.0)

In [50]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0558081852005  0.052662037037
 Accuracy Rate    0.9441918148  0.947337962963
True Positives 0.0684410646388 0.0627802690583
True Negatives              --  0.987099375126


(0.052662037037037035,
 0.94733796296296302,
 0.062780269058295965,
 0.98709937512598267)

In [51]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.434642414221 0.434992283951
 Accuracy Rate 0.565357585779 0.565007716049
True Positives  0.98288973384 0.968609865471
True Negatives             --   0.5468655513


(0.43499228395061729,
 0.56500771604938271,
 0.96860986547085204,
 0.54686555130014114)

# Happy/Calm 

In [52]:
# define y
y = totalData.haca.values
y.shape

(17279,)

In [53]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [54]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Happy/Calm  Model Results

In [55]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate  0.15221165771 0.149884259259
 Accuracy Rate  0.84778834229 0.850115740741
True Positives 0.316776007498 0.304147465438
True Negatives             --            0.9


(0.14988425925925927,
 0.8501157407407407,
 0.30414746543778803,
 0.90000000000000002)

In [56]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set      Test Set   
------------------- -------------- ---------------
         Error Rate 0.102025630426 0.0939429012346
      Accuracy Rate 0.897974369574  0.906057098765
     True Positives 0.204311152765  0.216589861751
     True Negatives             --  0.969052631579


(0.093942901234567902,
 0.90605709876543206,
 0.21658986175115208,
 0.96905263157894739)

In [57]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0915254237288 0.0868055555556
 Accuracy Rate   0.908474576271  0.913194444444
True Positives 0.00656044985942             0.0
True Negatives               --  0.996631578947


(0.086805555555555552, 0.91319444444444442, 0.0, 0.99663157894736842)

In [58]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0892930963208  0.0842978395062
 Accuracy Rate  0.910706903679   0.915702160494
True Positives             0.0 0.00230414746544
True Negatives              --   0.999157894737


(0.084297839506172839,
 0.91570216049382713,
 0.002304147465437788,
 0.99915789473684213)

# Anxious/Angry

In [59]:
# define y
y = totalData.anan.values
y.shape

(17279,)

In [60]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [61]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Angry Model Results

In [62]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0116577097974 0.0127314814815
 Accuracy Rate  0.988342290203  0.987268518519
True Positives             0.0             0.0
True Negatives              --  0.998634146341


(0.012731481481481481, 0.98726851851851849, 0.0, 0.99863414634146341)

In [63]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set   
------------------- ---------------- ---------------
         Error Rate 0.00950806118231 0.0113811728395
      Accuracy Rate   0.990491938818   0.98861882716
     True Positives              0.0             0.0
     True Negatives               --             1.0


(0.011381172839506173, 0.98861882716049387, 0.0, 1.0)

In [64]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00967341876809 0.0117669753086
 Accuracy Rate   0.990326581232  0.988233024691
True Positives              0.0             0.0
True Negatives               --  0.999609756098


(0.011766975308641976, 0.98823302469135799, 0.0, 0.99960975609756098)

In [65]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00950806118231 0.0115740740741
 Accuracy Rate   0.990491938818  0.988425925926
True Positives              0.0             0.0
True Negatives               --  0.999804878049


(0.011574074074074073, 0.98842592592592593, 0.0, 0.99980487804878049)

# Excited/Happy

In [66]:
# define y
y = totalData.exha.values
y.shape

(17279,)

In [67]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [68]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Excited/Happy Model Values

In [69]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0806945018603  0.0825617283951
 Accuracy Rate   0.91930549814   0.917438271605
True Positives 0.0021436227224 0.00246305418719
True Negatives              --   0.995186270406


(0.082561728395061734,
 0.91743827160493829,
 0.0024630541871921183,
 0.99518627040602758)

In [70]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0774700289376 0.0785108024691
      Accuracy Rate  0.922529971062  0.921489197531
     True Positives             0.0             0.0
     True Negatives              --  0.999790707409


(0.078510802469135804, 0.92148919753086422, 0.0, 0.9997907074089577)

In [71]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0812732534105  0.0829475308642
 Accuracy Rate  0.918726746589   0.917052469136
True Positives 0.0096463022508 0.00492610837438
True Negatives              --   0.994558392633


(0.082947530864197525,
 0.91705246913580252,
 0.0049261083743842365,
 0.9945583926329008)

In [72]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0780487804878 0.0790895061728
 Accuracy Rate  0.921951219512  0.920910493827
True Positives 0.0010718113612             0.0
True Negatives              --  0.999162829636


(0.07908950617283951, 0.9209104938271605, 0.0, 0.99916282963583092)

# Anxious/Sad

In [73]:
# define y
y = totalData.ansa.values
y.shape

(17279,)

In [74]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [75]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Sad Model Results

In [76]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate  0.0593633732947  0.0580632716049
 Accuracy Rate   0.940636626705   0.941936728395
True Positives 0.00578871201158 0.00344827586207
True Negatives               --   0.997548017981


(0.058063271604938273,
 0.94193672839506171,
 0.0034482758620689655,
 0.99754801798120152)

In [77]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.163455973543 0.161072530864
      Accuracy Rate 0.836544026457 0.838927469136
     True Positives 0.217076700434  0.21724137931
     True Negatives             -- 0.875766244381


(0.16107253086419754,
 0.83892746913580241,
 0.21724137931034482,
 0.87576624438087458)

In [78]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0571310458867 0.0559413580247
 Accuracy Rate  0.942868954113  0.944058641975
True Positives             0.0             0.0
True Negatives              --             1.0


(0.055941358024691357, 0.94405864197530864, 0.0, 1.0)

In [79]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0631665977677 0.0613425925926
 Accuracy Rate  0.936833402232  0.938657407407
True Positives 0.0347322720695  0.051724137931
True Negatives              --  0.991213731099


(0.061342592592592594,
 0.93865740740740744,
 0.051724137931034482,
 0.99121373109930522)